# NEW

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pandas as pd

class TransportationOptimizer:
    def __init__(self, root):
        self.root = root
        self.root.title("Transportation Problem Solver")
        self.root.geometry("700x500")

        # Mode selection
        self.mode = tk.StringVar(value="minimize")
        ttk.Label(self.root, text="Select Mode:").grid(row=0, column=0, padx=5, pady=5)
        ttk.Radiobutton(self.root, text="Minimize Cost", variable=self.mode, value="minimize").grid(row=0, column=1, padx=5)
        ttk.Radiobutton(self.root, text="Maximize Profit", variable=self.mode, value="maximize").grid(row=0, column=2, padx=5)

        # Input for rows and columns
        ttk.Label(self.root, text="Supply Constraints:").grid(row=1, column=0, padx=5, pady=5)
        self.supply_var = tk.IntVar(value=3)
        ttk.Entry(self.root, textvariable=self.supply_var, width=5).grid(row=1, column=1, padx=5)

        ttk.Label(self.root, text="Demand Constraints:").grid(row=1, column=2, padx=5)
        self.demand_var = tk.IntVar(value=4)
        ttk.Entry(self.root, textvariable=self.demand_var, width=5).grid(row=1, column=3, padx=5)

        ttk.Button(self.root, text="Generate", command=self.generate_matrix).grid(row=1, column=4, padx=5)

        # Method selection
        ttk.Label(self.root, text="Method:").grid(row=2, column=0, padx=5, pady=5)
        self.method_var = tk.StringVar(value="plus-trend")
        ttk.Combobox(self.root, textvariable=self.method_var, values=["plus-trend", "cross-trend"]).grid(row=2, column=1, padx=5)

        # Table placeholders
        self.table_frame = ttk.Frame(self.root)
        self.table_frame.grid(row=3, column=0, columnspan=5, padx=10, pady=10)

        ttk.Button(self.root, text="Compute", command=self.compute_solution).grid(row=4, column=0, columnspan=5, pady=10)

        self.result_label = ttk.Label(self.root, text="Final Cost: ")
        self.result_label.grid(row=5, column=0, columnspan=5, pady=10)

    def generate_matrix(self):
        for widget in self.table_frame.winfo_children():
            widget.destroy()

        self.entries = []
        self.supply_entries = []
        self.demand_entries = []
        supply = self.supply_var.get()
        demand = self.demand_var.get()

        # Header Row
        ttk.Label(self.table_frame, text="").grid(row=0, column=0)
        for j in range(demand):
            ttk.Label(self.table_frame, text=f"D{j+1}", padding=5).grid(row=0, column=j+1)
        ttk.Label(self.table_frame, text="Supply", padding=5).grid(row=0, column=demand+1)

        # Data Input Matrix with S1, S2, ... labels and supply values
        for i in range(supply):
            ttk.Label(self.table_frame, text=f"S{i+1}", padding=5).grid(row=i+1, column=0)
            row_entries = []
            for j in range(demand):
                entry = ttk.Entry(self.table_frame, width=5)
                entry.grid(row=i+1, column=j+1, padx=2, pady=2)
                row_entries.append(entry)
            supply_entry = ttk.Entry(self.table_frame, width=5)
            supply_entry.grid(row=i+1, column=demand+1, padx=2, pady=2)
            self.supply_entries.append(supply_entry)
            self.entries.append(row_entries)

        # Demand Row
        ttk.Label(self.table_frame, text="Demand", padding=5).grid(row=supply+1, column=0)
        for j in range(demand):
            entry = ttk.Entry(self.table_frame, width=5)
            entry.grid(row=supply+1, column=j+1, padx=2, pady=2)
            self.demand_entries.append(entry)

    def compute_solution(self):
        try:
            supply = self.supply_var.get()
            demand = self.demand_var.get()
            cost_matrix = np.zeros((supply, demand))
            supply_values = []
            demand_values = []

            for i in range(supply):
                for j in range(demand):
                    cost_matrix[i][j] = float(self.entries[i][j].get())
                supply_values.append(float(self.supply_entries[i].get()))

            for j in range(demand):
                demand_values.append(float(self.demand_entries[j].get()))

            # Verify supply equals demand
            if sum(supply_values) != sum(demand_values):
                messagebox.showerror("Error", "Total supply must equal total demand!")
                return

            method = self.method_var.get()
            if method == "plus-trend":
                solution, final_cost = self.optimize_plus_trend(cost_matrix, supply_values, demand_values)
            else:
                solution, final_cost = self.optimize_cross_trend(cost_matrix, supply_values, demand_values)

            self.result_label.config(text=f"Final Cost: {final_cost}")
            print("Allocation Matrix:", solution)  # For debugging
        except ValueError as e:
            messagebox.showerror("Error", "Invalid input, please enter numeric values only.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def optimize_plus_trend(self, cost_matrix, supply, demand):
        m, n = cost_matrix.shape
        allocation = np.zeros((m, n))
        final_cost = 0

        row_mid = m // 2
        col_mid = n // 2
        
        indices = [(row_mid, j) for j in range(n)] + [(i, col_mid) for i in range(m)]
        indices = list(dict.fromkeys(indices))  # Remove duplicates
        indices.sort(key=lambda x: cost_matrix[x[0], x[1]])

        for i, j in indices:
            if supply[i] > 0 and demand[j] > 0:
                min_val = min(supply[i], demand[j])
                allocation[i][j] = min_val
                final_cost += min_val * cost_matrix[i][j]
                supply[i] -= min_val
                demand[j] -= min_val

        for i in range(m):
            for j in range(n):
                if supply[i] > 0 and demand[j] > 0:
                    min_val = min(supply[i], demand[j])
                    allocation[i][j] += min_val
                    final_cost += min_val * cost_matrix[i][j]
                    supply[i] -= min_val
                    demand[j] -= min_val

        return allocation.tolist(), final_cost

    def optimize_cross_trend(self, cost_matrix, supply, demand):
        m, n = cost_matrix.shape
        allocation = np.zeros((m, n))
        final_cost = 0

        while np.sum(supply) > 0 and np.sum(demand) > 0:
            min_cost = np.inf
            min_i, min_j = -1, -1
            
            for i in range(m):
                for j in range(n):
                    if supply[i] > 0 and demand[j] > 0 and cost_matrix[i][j] < min_cost:
                        min_cost = cost_matrix[i][j]
                        min_i, min_j = i, j

            if min_i == -1 or min_j == -1:
                break

            min_val = min(supply[min_i], demand[min_j])
            allocation[min_i][min_j] = min_val
            final_cost += min_val * cost_matrix[min_i][min_j]
            supply[min_i] -= min_val
            demand[min_j] -= min_val

        return allocation.tolist(), final_cost

if __name__ == "__main__":
    root = tk.Tk()
    app = TransportationOptimizer(root)
    root.mainloop()

Allocation Matrix: [[140.0, 0.0, 60.0], [40.0, 120.0, 0.0], [0.0, 0.0, 90.0]]
Allocation Matrix: [[50.0, 0.0, 150.0], [40.0, 120.0, 0.0], [90.0, 0.0, 0.0]]
Allocation Matrix: [[5.0, 0.0, 0.0, 0.0], [0.0, 8.0, 0.0, 2.0], [0.0, 0.0, 7.0, 0.0], [0.0, 0.0, 3.0, 0.0]]
Allocation Matrix: [[0.0, 5.0, 0.0, 0.0], [5.0, 3.0, 0.0, 2.0], [0.0, 0.0, 7.0, 0.0], [0.0, 0.0, 3.0, 0.0]]
Allocation Matrix: [[5.0, 6.0, 0.0, 0.0], [1.0, 0.0, 12.0, 0.0], [0.0, 4.0, 0.0, 15.0]]
Allocation Matrix: [[0.0, 0.0, 0.0, 11.0], [1.0, 0.0, 12.0, 0.0], [5.0, 10.0, 0.0, 4.0]]
Allocation Matrix: [[0.0, 42.0, 0.0, 108.0, 0.0], [0.0, 0.0, 136.0, 92.0, 82.0], [100.0, 120.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 185.0]]
Allocation Matrix: [[0.0, 0.0, 0.0, 0.0, 150.0], [0.0, 0.0, 136.0, 174.0, 0.0], [100.0, 120.0, 0.0, 0.0, 0.0], [0.0, 42.0, 0.0, 26.0, 117.0]]
